In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
#from utee import misc, quant, selector

import torch.nn.functional as F  # useful stateless functions

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np


#Load CIFAR-10
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [3]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [4]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    t_begin = time.time()
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                t_elapse = time.time() - t_begin
                print('Elapsed %.4f s, Epoch %d,  Iteration %d, loss = %.4f' % (t_elapse, e, t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [5]:
from Model.quantization import *

#TODO: 1. Implement different function
#      2. differentiate FC and CONV
def quantize_weight(model, bits):
    weight = model.state_dict()
    for k, v in weight.items():
        weight[k] = Quant.linear(v, bits)
        #print(weight[k])
    model.load_state_dict(weight)
    return model

#TODO: Add a dictionary for bit width and function.
def train_fixed_weight(model, optimizer, epochs=1, bits=8):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    t_begin = time.time()
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            
            #Add quantization for weight
            model = quantize_weight(model, bits)

            if t % print_every == 0:
                t_elapse = time.time() - t_begin
                print('Elapsed %.4f s, Epoch %d,  Iteration %d, loss = %.4f' % (t_elapse, e, t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [6]:
from Model.vgg_modules import *

VGG19 = vgg19()
VGG19.state_dict().keys()

odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.16.weight', 'features.16.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.23.weight', 'features.23.bias', 'features.25.weight', 'features.25.bias', 'features.28.weight', 'features.28.bias', 'features.30.weight', 'features.30.bias', 'features.32.weight', 'features.32.bias', 'features.34.weight', 'features.34.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])

In [7]:
PATH = '../pretrain_model/model_best.pth.tar'
VGG19.features = torch.nn.DataParallel(VGG19.features)
VGG19.cuda()
checkpoint = torch.load(PATH)
start_epoch = checkpoint['epoch']
best_prec1 = checkpoint['best_prec1']
#print(VGG19.state_dict().keys())
#print(checkpoint['state_dict'].keys())
VGG19.load_state_dict(checkpoint['state_dict'])

In [33]:
from Model.Fixedvgg import *

exp_bit = 16

FixedVGG19 = fixed_vgg19(bits=exp_bit)
FixedVGG19

PATH = '../pretrain_model/model_best.pth.tar'
FixedVGG19.features = torch.nn.DataParallel(FixedVGG19.features)
FixedVGG19.cuda()
checkpoint = torch.load(PATH)
start_epoch = checkpoint['epoch']
best_prec1 = checkpoint['best_prec1']
#print(VGG19.state_dict().keys())
#print(checkpoint['state_dict'].keys())
FixedVGG19.load_state_dict(checkpoint['state_dict'])

FixedVGG19 = quantize_weight(FixedVGG19, exp_bit)

In [34]:
print("\nFixed VGG19 Accuracy:")
check_accuracy_part34(loader_test, FixedVGG19)
print("\nFloat VGG19 Accuracy:")
check_accuracy_part34(loader_test, VGG19)


Fixed VGG19 Accuracy:
Checking accuracy on test set
Got 9205 / 10000 correct (92.05)

Float VGG19 Accuracy:
Checking accuracy on test set
Got 9206 / 10000 correct (92.06)


In [35]:
#Train this model
learning_rate = 4e-5

optimizer = optim.Adam(params=FixedVGG19.parameters(), lr=learning_rate)
#optimizer = optim.SGD(FixedVGG19.parameters(), lr=learning_rate,momentum=0.9, nesterov=True)
train_fixed_weight(FixedVGG19, optimizer, epochs=10, bits=exp_bit)

Elapsed 0.0596 s, Epoch 0,  Iteration 0, loss = 0.0008
Checking accuracy on validation set
Got 996 / 1000 correct (99.60)

Elapsed 6.2508 s, Epoch 0,  Iteration 100, loss = 0.0011
Checking accuracy on validation set
Got 997 / 1000 correct (99.70)

Elapsed 12.4009 s, Epoch 0,  Iteration 200, loss = 0.0069
Checking accuracy on validation set
Got 998 / 1000 correct (99.80)

Elapsed 18.5653 s, Epoch 0,  Iteration 300, loss = 0.0069
Checking accuracy on validation set
Got 997 / 1000 correct (99.70)

Elapsed 24.7219 s, Epoch 0,  Iteration 400, loss = 0.0007
Checking accuracy on validation set
Got 977 / 1000 correct (97.70)

Elapsed 30.8803 s, Epoch 0,  Iteration 500, loss = 0.0010
Checking accuracy on validation set
Got 998 / 1000 correct (99.80)

Elapsed 37.0349 s, Epoch 0,  Iteration 600, loss = 0.0552
Checking accuracy on validation set
Got 995 / 1000 correct (99.50)

Elapsed 43.2028 s, Epoch 0,  Iteration 700, loss = 0.0002
Checking accuracy on validation set
Got 996 / 1000 correct (99.6

Elapsed 384.4951 s, Epoch 8,  Iteration 100, loss = 0.0006
Checking accuracy on validation set
Got 993 / 1000 correct (99.30)

Elapsed 390.6342 s, Epoch 8,  Iteration 200, loss = 0.0023
Checking accuracy on validation set
Got 988 / 1000 correct (98.80)

Elapsed 396.8240 s, Epoch 8,  Iteration 300, loss = 0.0070
Checking accuracy on validation set
Got 990 / 1000 correct (99.00)

Elapsed 403.0384 s, Epoch 8,  Iteration 400, loss = 0.0415
Checking accuracy on validation set
Got 989 / 1000 correct (98.90)

Elapsed 409.2592 s, Epoch 8,  Iteration 500, loss = 0.0023
Checking accuracy on validation set
Got 987 / 1000 correct (98.70)

Elapsed 415.4775 s, Epoch 8,  Iteration 600, loss = 0.0006
Checking accuracy on validation set
Got 995 / 1000 correct (99.50)

Elapsed 421.7002 s, Epoch 8,  Iteration 700, loss = 0.1264
Checking accuracy on validation set
Got 992 / 1000 correct (99.20)

Elapsed 425.9076 s, Epoch 9,  Iteration 0, loss = 0.0013
Checking accuracy on validation set
Got 993 / 1000 cor

In [32]:
print("\nFloat VGG19 Accuracy:")
check_accuracy_part34(loader_test, VGG19)
print("\nFinetune Fixed VGG19 Accuracy:")
check_accuracy_part34(loader_test, FixedVGG19)



Float VGG19 Accuracy:
Checking accuracy on test set
Got 9206 / 10000 correct (92.06)

Finetune Fixed VGG19 Accuracy:
Checking accuracy on test set
Got 9203 / 10000 correct (92.03)
